In [1]:
import sys
import os
from pathlib import Path

%reload_ext autoreload
%autoreload 2

base_path=Path("d:/")
base_path = base_path / "work" / "projects"
for pkg in ['ezai_util','ai_coop_py']:
    pkg_path = base_path / pkg
    pkg_path = str(pkg_path.resolve())
    print(pkg_path)
    if not pkg_path in sys.path:
        sys.path.append(pkg_path)
import ezai_util
import airsim

D:\work\projects\ezai_util
D:\work\projects\ai_coop_py


In [2]:
uenv_path = Path("d:/") / 'work' / 'unity-envs'
uenv_file = uenv_path / "airsim_win" / 'AirSimAssets'
uenv_file = str(uenv_file.resolve())

In [3]:
env = airsim.AirSimEnv(filename=uenv_file, observation_mode = 0)

2020-10-11 22:54:05 INFO [environment.py:107] Connected to Unity environment with package version 1.1.0-preview and communication version 1.0.0
2020-10-11 22:54:05 INFO [environment.py:265] Connected new brain:
VisualNavigator?team=0
c:\Miniconda3\envs\ezai\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [4]:
env.info()

Env Info
-----------
Action Space: Box(2,)
Action sample: [ 0.39293838 -0.42772132]
Action Space Shape: (2,)
Action Space Low: [-1. -1.]
Action Space High: [1. 1.]
Observation Mode: 0
Gym Observation Space: Tuple(Box(10,))
Gym Observation Space Shape: None
Self Observation Space: Tuple(Box(10,))
Self Observation Space Shape: None
Reward Range: (-inf, inf)
Metadata: {'render.modes': ['rgb_array']}
Initial State: [array([ 2.3926267e+03,  3.7319370e+01, -3.7768066e+02, -2.9097432e-03,
        9.2058080e-01,  3.1531379e-03, -3.9052874e-01,  2.5062500e+03,
        4.1240845e+01, -3.6359070e+02], dtype=float32)]
First Step State: ([array([ 2.3926272e+03,  3.7317383e+01, -3.7768079e+02, -3.7698867e-03,
        9.2058468e-01,  5.0607547e-03, -3.9049214e-01,  2.5062500e+03,
        4.1240845e+01, -3.6359070e+02], dtype=float32)], -1e-04, False, {'step': <mlagents_envs.base_env.DecisionSteps object at 0x00000208C66BCE20>})


In [5]:
from ezai_util import DictObj

In [17]:
model_conf = DictObj()
model_conf.n_x = env.n_x
model_conf.n_y = env.n_y
model_conf.n_batch_size = 64
model_conf.n_epochs = 10

policy_conf = DictObj()
policy_conf.discount_rate=0.9
policy_conf.learning_rate=0.8
policy_conf.explore_rate_max=0.9
policy_conf.explore_rate_min=0.01
policy_conf.explore_rate_decay=0.005
policy_conf.memory_capacity = model_conf.n_batch_size * 10

rl_conf = DictObj({
  "env_name":'airsim',
  "seed": 0,
  "discount": 0.99,
  "tau": 5e-3,
  "time_steps": 100,
  "start_time_step": 25e3,
  "expl_noise": 0.1,
  "batch_size": 256,
  "evaluate_frequency": 5e3,
  "max_episode_steps":200000,
  "memory_capacity":10000
})

ezai - I - Making empty DictObj because not given a dict or a json filename
2020-10-11 23:10:17 INFO [dict.py:41] Making empty DictObj because not given a dict or a json filename
ezai - I - Making empty DictObj because not given a dict or a json filename
2020-10-11 23:10:17 INFO [dict.py:41] Making empty DictObj because not given a dict or a json filename


In [18]:
trainer = airsim.Trainer(rl_conf,env)

2020-10-11 23:10:19 WARNING [__init__.py:270] Could not seed environment VisualNavigator?team=0


(10,)


In [19]:
episode_rewards, evaluations = trainer.train()

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 270.55it/s]


In [41]:
print(env.reset()[0].shape)

(7,)


In [42]:

trainer.agent.save_actor('airsim.onnx'\\)

TypeError: save_actor() takes 2 positional arguments but 3 were given

In [20]:
env.close()

[autoreload of airsim.classes failed: Traceback (most recent call last):
  File "c:\Miniconda3\envs\ezai\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "c:\Miniconda3\envs\ezai\lib\site-packages\IPython\extensions\autoreload.py", line 394, in superreload
    module = reload(module)
  File "c:\Miniconda3\envs\ezai\lib\imp.py", line 314, in reload
    return importlib.reload(module)
  File "c:\Miniconda3\envs\ezai\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 779, in exec_module
  File "<frozen importlib._bootstrap_external>", line 916, in get_code
  File "<frozen importlib._bootstrap_external>", line 846, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "D:\work\projects\ai_coop_py\airsim\classes.py", line 158

UnityEnvironmentException: No Unity environment is loaded.

In [ ]:
model = airsim.AirSimNN(model_conf)
model.info()

In [ ]:
# create a memory of random play

from ezai_rl.core import Memory
from ezai_rl.policies import RandomPolicy

memory = Memory(policy_conf.memory_capacity)
policy = RandomPolicy(env.action_space)

t_max = policy_conf.memory_capacity
t=0
episode_done = False

#1. observe initial state
s = env.reset()

while not episode_done:
    #2. select an action, and observe the next state
    a = policy.get_action(s)
    s_, r, episode_done, info = env.step(a)
    
    if(t == t_max-1):
        episode_done=True
    
    #3. save in memory

    #s = np.asarray(s).squeeze()
    #a = np.asarray(a).squeeze()
    #r = np.asarray(r).squeeze()
    s_ = None if episode_done else s_ #np.asarray(s_).squeeze()
    memory.append([s,a,r,s_])
    
    # update the counters
    t += 1
    
    if (t_max and t >= t_max):
        break

"""    
a=[]
s=[]
for i in range(1000):
    a_i = env.action_space.sample()
    s_i = env.step(a_i)[0] if i else env.reset()
    a.append(a_i)
    s.append(s_i)

a=np.asarray(a).squeeze()
s=np.asarray(s).squeeze()
print('actions \n',a.shape,a[0])
print('states \n',s.shape,s[0])
"""

In [ ]:
memory[-2:].pretty_print()

In [ ]:
sample = memory.sample(0)
ss, aa, rr, ss_ = zip(*sample)
nt_mask = tuple(map(lambda st: st is not None,ss_))
print(nt_mask[-2:0])

In [ ]:
# test the model
model.fit(x_train=s,y_train=a)

last_state = s[-1]
a=[]
s=[]
for i in range(10):
    a_i = model.predict(last_state)
    s_i = env.step(a_i)[0]
    last_state = np.asarray(s_i).squeeze()
    a.append(a_i)
    s.append(s_i)

a=np.asarray(a).squeeze()
s=np.asarray(s).squeeze()
print('actions \n',a.shape,a)
print('states \n',s.shape,s)

In [ ]:
from ezai_rl.policies import DQNPolicy
from ezai_rl.policies import RandomPolicy
from ezai_rl.trial import run_episodes

In [ ]:
policy_conf = DictObj()
policy_conf.discount_rate=0.9
policy_conf.learning_rate=0.8
policy_conf.explore_rate_max=0.9
policy_conf.explore_rate_min=0.01
policy_conf.explore_rate_decay=0.005
policy_conf.memory_capacity = model_conf.n_batch_size * 10

In [ ]:
# collect observations and actions from random_policy
policy_rand = RandomPolicy(env.action_space)


In [ ]:
#policy_rand = RandomPolicy(env.action_space)
policy_dqn = DQNPolicy(env.action_space, env.observation_space, policy_conf, model)

print('training policies')
n_episodes = 1 #000
returns=run_episodes(env,policy_dqn, n_episodes, learn_mode=True)

print('using trained policies')
n_episodes=10
returns=run_episodes(env,policy_rand,n_episodes)
#returns=run_episodes(env,policy_qt,n_episodes)

In [ ]:
env.close()

In [ ]:
# test action_space types
uenv = UnityEnvironment(file_name=uenv_file, 
                        log_folder='/tmp/unity', seed = 123, 
                        side_channels =[engine_side_channel,environment_side_channel] )
engine_side_channel.set_configuration_parameters(time_scale=10, quality_level = 0)

#uenv.reset()
genv = UnityToGymWrapper(uenv, False, False, True)#(Environment, uint8_visual, flatten_branched, allow_multiple_obs)
obs = genv.reset()

print(type(genv.action_space))
print(type(genv.action_space.sample()))
print(genv.action_space)
print(genv.action_space.sample())

genv.close()

In [ ]:
# test sample() randomization

for i in range(5):
    print('run ',i)
    uenv = UnityEnvironment(file_name=uenv_file, 
                            log_folder='/tmp/unity', seed = 123, 
                            side_channels =[engine_side_channel,environment_side_channel] )
    engine_side_channel.set_configuration_parameters(time_scale=10, quality_level = 0)

    #uenv.reset()
    genv = UnityToGymWrapper(uenv, False, False, True)#(Environment, uint8_visual, flatten_branched, allow_multiple_obs)
    genv.action_space.np_random.seed(123)
    obs = genv.reset()

    #print(ezai_rl.env.env_info(genv))
    for j in range(5):
        print(genv.action_space.sample())

    genv.close()

In [ ]:
uenv = UnityEnvironment(file_name=uenv_file, 
                        log_folder='/tmp/unity', seed = 1, 
                        side_channels =[engine_side_channel,environment_side_channel] )
engine_side_channel.set_configuration_parameters(time_scale=10, quality_level = 0)
environment_side_channel.set_float_parameter("rewardForGoalCollision", .5)
environment_side_channel.set_float_parameter("rewardForExplorationPointCollision", .005)
environment_side_channel.set_float_parameter("rewardForOtherCollision", -.1)
environment_side_channel.set_float_parameter("rewardForFallingOffMap", -1)
environment_side_channel.set_float_parameter("rewardForEachStep", -.0001)
environment_side_channel.set_float_parameter("segmentationMode", 1)
#uenv.reset()
genv = UnityToGymWrapper(uenv, True, False, True)#(Environment, uint8_visual, flatten_branched, allow_multiple_obs)
obs = genv.reset()

print(ezai_rl.core.env_info(genv))

#genv.close()
#uenv.close()

In [ ]:
env = genv

In [ ]:
print(env.observation_space.n)

In [ ]:
from ezai_rl.policies import DQNPolicy
from ezai_rl.policies import RandomPolicy
from ezai_rl.trial import run_episodes

In [ ]:
# remove the seeds to get random plays everytime
np.random.seed(123)
env.seed(123) # doesnt work on unity
env.action_space.np_random.seed(123)

policy_rand = RandomPolicy(env.action_space)
policy_dqn = DQNPolicy(env.action_space, env.observation_space)

print('training policies')
n_episodes = 1 #000
returns=run_episodes(env,policy_dqn, n_episodes, learn_mode=True)

print('using trained policies')
n_episodes=10
returns=run_episodes(env,policy_rand,n_episodes)
#returns=run_episodes(env,policy_qt,n_episodes)

In [53]:
env.close()

2020-10-09 19:05:32 INFO [environment.py:417] Environment shut down with return code 0 (CTRL_C_EVENT).


In [21]:
a = [5,6]

In [22]:
print(a)

[5, 6]


In [23]:
a = [a]

In [24]:
print(a)

[[5, 6]]


In [25]:
import numpy as np


In [30]:
arr = np.full((tuple(2) + (5,2)), 0.0)
print(arr)

TypeError: unsupported operand type(s) for +: 'int' and 'tuple'

In [52]:
a=2 + list([2,3])
#a.insert(0,1)
print(a)

TypeError: unsupported operand type(s) for +: 'int' and 'list'